In [ ]:
# load audio file Cenicham
# [x, fs] = audioread('TwoChannelMic.wav')

# in pyhon
import numpy as np
import soundfile as sf
from scipy.signal import correlate
from scipy.constants import speed_of_sound
import plotly.graph_objs as go

x_mysgn, fs = sf.read('Cenicham_zkracene_mastered_hiphop_loud.wav')

# only take first 5 seconds
x_mysgn = x_mysgn[:5*fs]

# print duration of signal in seconds
T = len(x_mysgn) / fs
# high pass filter y

#filtered_signal = np.zeros(len(y))

#fs = 1000  # Sampling frequency (Hz)
fc = 3000    # Cutoff frequency (Hz)
#T = 1.0    # Duration in seconds
alpha = 2 * np.pi * fc / (2 * np.pi * fc + fs)


# high pass filter
#y = np.zeros_like(x)
#for n in range(1, len(x)):
#    y[n] = x[n] - x[n-1] + alpha * y[n-1]

#  matlab filter designer
# B = fir1(100,3000/(fs)/2,'high')

In [ ]:
from scipy.io.wavfile import write

# Save the filtered audio as a temporary WAV file
output_filename = 'filtered_audio.wav'
write(output_filename, fs, np.int16(y * 32767))

# Play the saved audio file
from playsound import playsound
playsound(output_filename)


In [ ]:
from scipy import signal
cutoff = 3000
normalized_cutoff = cutoff / (fs / 2)
order = 100

B = signal.firwin(100 + 1, normalized_cutoff, pass_zero=False)


# Frequency response
w, h = signal.freqz(B, worN=8000)
frequencies = w * fs / (2 * np.pi)  # Convert from rad/sample to Hz

In [ ]:
# Plot frequency response using Plotly
magnitude_response = 20 * np.log10(abs(h))
trace = go.Scatter(x=frequencies, y=magnitude_response, mode='lines', name='Magnitude Response')

layout = go.Layout(
    title='High-pass FIR Filter Frequency Response',
    xaxis=dict(title='Frequency (Hz)', type='linear'),
    yaxis=dict(title='Magnitude (dB)'),
    showlegend=True,
    hovermode='x unified',
    template='plotly_dark',
    shapes=[
        dict(
            type='line',
            x0=cutoff, y0=-80, x1=cutoff, y1=0,
            line=dict(color='red', dash='dash')
        )
    ],
    annotations=[
        dict(
            x=cutoff,
            y=-40,
            xref="x",
            yref="y",
            text="Cutoff Frequency (3000 Hz)",
            showarrow=True,
            arrowhead=7,
            ax=40,
            ay=40
        )
    ]
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [ ]:
# Design the high-pass FIR filter using firwin()
B = signal.firwin(order + 1, normalized_cutoff, pass_zero=False)

# Plot the filter coefficients (Impulse Response)
trace = go.Scatter(
    x=np.arange(len(B)),
    y=B,
    mode='lines+markers',
    name='Filter Coefficients'
)

layout = go.Layout(
    title='FIR High-pass Filter Coefficients (Impulse Response)',
    xaxis=dict(title='Coefficient Index'),
    yaxis=dict(title='Amplitude'),
    showlegend=True,
    hovermode='x unified',
    template='plotly_dark'
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [ ]:
# filter x with B
y = signal.lfilter(B, 1, x_mysgn)

# play the filtered signal
output_filename = 'filtered_audio.wav'
write(output_filename, fs, np.int16(y * 32767))
playsound(output_filename)

In [ ]:
# Apply the high-pass filter to the signal x
y = signal.lfilter(B, 1.0, x)

# Plotting the Original and Filtered Signals Using Plotly
trace_original = go.Scatter(
    x=t,
    y=x,
    mode='lines',
    name='Original Signal',
    line=dict(color='blue')
)

trace_filtered = go.Scatter(
    x=t,
    y=y,
    mode='lines',
    name='Filtered Signal (High-pass)',
    line=dict(color='orange')
)

layout = go.Layout(
    title='Original and High-pass Filtered Signal',
    xaxis=dict(title='Time (seconds)'),
    yaxis=dict(title='Amplitude'),
    showlegend=True,
    hovermode='x unified',
    template='plotly_dark'
)

fig = go.Figure(data=[trace_original, trace_filtered], layout=layout)
fig.show()

In [ ]:
# oprava, stereo!!!!!!!


import numpy as np
import soundfile as sf
from scipy import signal
import plotly.graph_objs as go

# Load the audio file (replace with your file path)
x, fs = sf.read('Cenicham_zkracene_mastered_hiphop_loud.wav')

# Check if the audio is stereo or mono
if len(x.shape) == 2:
    print("Stereo audio detected.")
    x_left = x[:, 0]   # Left channel
    x_right = x[:, 1]  # Right channel
else:
    print("Mono audio detected.")
    x_left = x
    x_right = None

# Only take the first 5 seconds
x_left = x_left[:5 * fs]
if x_right is not None:
    x_right = x_right[:5 * fs]

# Time axis
t = np.linspace(0, len(x_left) / fs, num=len(x_left))

# Design a High-pass FIR Filter (similar to MATLAB's fir1())
order = 100
cutoff = 3000
normalized_cutoff = cutoff / (fs / 2)  # Proper normalization

# Filter coefficients
B = signal.firwin(order + 1, normalized_cutoff, pass_zero=False)

# Filter the left channel
y_left = signal.lfilter(B, 1.0, x_left)

# Filter the right channel if stereo
if x_right is not None:
    y_right = signal.lfilter(B, 1.0, x_right)

# Plot the Original and Filtered Signal (Left Channel) Using Plotly
trace_original = go.Scatter(
    x=t,
    y=x_left,
    mode='lines',
    name='Original Left Channel',
    line=dict(color='blue')
)

trace_filtered = go.Scatter(
    x=t,
    y=y_left,
    mode='lines',
    name='Filtered Left Channel',
    line=dict(color='orange')
)

# If stereo, plot the right channel too
if x_right is not None:
    trace_original_right = go.Scatter(
        x=t,
        y=x_right,
        mode='lines',
        name='Original Right Channel',
        line=dict(color='green')
    )

    trace_filtered_right = go.Scatter(
        x=t,
        y=y_right,
        mode='lines',
        name='Filtered Right Channel',
        line=dict(color='red')
    )

layout = go.Layout(
    title='Original and Filtered Sound Waveform',
    xaxis=dict(title='Time (seconds)'),
    yaxis=dict(title='Amplitude'),
    showlegend=True,
    hovermode='x unified',
    template='plotly_dark'
)

# Add traces to the figure
fig = go.Figure(data=[trace_original, trace_filtered])
if x_right is not None:
    fig.add_trace(trace_original_right)
    fig.add_trace(trace_filtered_right)

fig.show()


In [ ]:
# signal + signal filtered with high pass filter -> amp up high frequencies
# ale posune se o par vzorku, nutno posunout, pridat par nul na zacatek x/2 (puvodni signal delime dvema pro snizeni hlasitosti)

# lze to odhalit vykreslenim charakteristiky filtru.
# spravne scitani fazi, kontrola prenosove funkce celeho systemu

In [ ]:
# Save the filtered audio as a temporary WAV file

output_filename = 'filtered_audio.wav'
sf.write(output_filename, y_left, fs)

# Play the saved audio file
playsound(output_filename)
